## Accessing SQL via Python
One reason we've been using SQLite databases is because of how well they work with Python. In this exercise we'll create databases, build tables, fill them with data, and write queries. Before we get started, make sure this notebook is saved somewhere sensible (like in a topic-specific folder) and move one of the Wedge transaction files into the same folder. I've included one of mine so the code will run as written, but I _urge_ you to use one of yours.

In [2]:
import sqlite3

working_dir = "C:/Users/jchan/Dropbox/Teaching/AppliedDataAnalytics/Code/sql-and-python/"
C:\Users\jchan\Dropbox\Teaching\AppliedDataAnalytics\Code\sql-and-python

input_file = "OwnerTransactions_30.txt"

# Let's just open the file and read the first few lines to the screen.
with open(working_dir + input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split("\t"))
        if idx > 3 :
            break

['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id']
['2010-01-01 10:19:32', '6', '68', '2', '2.79DP9', 'GEN MERCH', 'D', ' ', ' ', '9', '2', '0', '0.0000', '2.7900', '5.5800', '2.7900', '0.0000', '1', '0', '0', '0', '0.0000', '0.0000', '3', '0', '0', '0.00000000', '2', '0', '0', '0.0000', '0', '0', '', '0', '0', '0', '0', '0', '0', 'NULL', '0', 'NULL', '', '0', '40825', '1', '0', '0', '1']
['2010-01-01 10:19:46', '6', '68', '2', '0000000

Lots of fields (columns) in these files, most of them should look familiar by now. 

Let's create a database on your computer first. Change the name to something you like.

In [3]:
 db = sqlite3.connect(working_dir + "change_me.db")

Go check out that file in your folder with your favorite SQLite browser. You'll see an empty database with no tables. Let's create a transaction table in there. First, to do that, we need to set up a way to talk to the database. When we're talking to text files we use a filehandle (which we get by calling `open`). To talk to a database we need a cursor. Let's open one up.

In [3]:
cur = db.cursor()

Pretty straightforward. Now things get more complicated. Let's make a simple, empty table in the database. 

In [4]:
cur.execute('''DROP TABLE IF EXISTS test_table''')
cur.execute('''CREATE TABLE test_table (
    first_name TEXT, 
    last_name TEXT, 
    age INTEGER,
    height REAL)''')

Remember from the part of the lecture on temporary tables how we would drop a table before creating it? If you do that and the table doesn't exist, you get an error. That's annoying in this code, so we use the `DROP TABLE IF EXISTS` syntax. Also notice how we use the triple quotes. This isn't necessary, but it allows us to format our SQL the same way we would in the query engine, which makes it more readable. Finally, notice that we have to declare the colume type for each variable. Here are the main types in SQLite:
* INTEGER: A signed integer which can be up to 8 bytes ($2^{64}$ = 18,446,744,073,709,551,615)
* REAL: A floating point value.
* TEXT: A text string.
* TIMESTAMP: Not a native type, but you will call datetime functions to get you something that works the way you want it to.

Now go look at the DB and you'll see our empty table. Let's put some records in it. 

In [5]:
cur.execute('''INSERT INTO test_table (first_name, last_name, age, height) 
               VALUES ('John','Chandler',43,6.42)''')

Notice the weird syntax. This is just something you have to know is happening when you write insert statements. I typically have to Google it if it's been a while. 

If you go look at the table, though, you won't see my row. That's because we have to tell the db to _commit_ the changes. Commit has a technical definition, but you can think of it as just making the change official.

In [6]:
db.commit()

In general you can just commit changes when you're doing working on a table and usually be fine. (Particularly if you're the only person using a DB.)

If you run the above two cells a few times, you can add multiple of my records to the DB. Feel free to do that.

Typing in records is lame. Let's use a list of lists do it. 

In [7]:
people = [["Ben","Chandler",35,6.14],
          ["Davis","Chandler",33,5.5],
          ["Martha","Chandler",73,5.67],
          ["Beth","Tanis",60,5.7]]

# SQLite3 gives you a fast way to put all these records in.
cur.executemany('''INSERT INTO test_table (first_name, last_name, age, height) 
                   VALUES (?,?,?,?)''',people)
db.commit()

Again, some weird syntax, but it is what it is. The handy function `executemany` can take any iterable in the second argument. As long as the "shape" of that second argument is correct, this will fill up your table. Now let's get rid of that goofy table. 

In [8]:
cur.execute('DROP TABLE test_table')
db.commit()

And, lo, our table is gone. If we wanted to we could now call `db.close()` and be done with it. We'll keep it open so we can put our transactions table in it.

### Working with the Wedge Data
These transaction tables are pretty massive, so I'm going to give you a version of the code that will build the table. Let the record reflect I'm saving you a bunch of tedious typing and error checking.

In [1]:
cur.execute('''DROP TABLE IF EXISTS transactions''')
cur.execute('''CREATE TABLE transactions (
    datetime TIMESTAMP,
    register_no INTEGER,
    emp_no INTEGER,
    trans_no INTEGER,
    upc INTEGER,
    description TEXT,
    trans_type TEXT,
    trans_subtype TEXT,
    trans_status TEXT,
    department INTEGER,
    quantity REAL,
    Scale INTEGER,
    cost REAL,
    unitPrice REAL,
    total REAL,
    regPrice REAL,
    altPrice REAL,
    tax INTEGER,
    taxexempt INTEGER,
    foodstamp INTEGER,
    wicable INTEGER,
    discount REAL,
    memDiscount REAL,
    discountable INTEGER,
    discounttype INTEGER,
    voided INTEGER,
    percentDiscount REAL,
    ItemQtty REAL,
    volDiscType INTEGER,
    volume INTEGER,
    VolSpecial REAL,
    mixMatch INTEGER,
    matched INTEGER,
    memType INTEGER,
    staff INTEGER,
    numflag INTEGER,
    itemstatus INTEGER,
    tenderstatus INTEGER,
    charflag TEXT,
    varflag INTEGER,
    batchHeaderID INTEGER,
    local INTEGER,
    organic INTEGER,
    display INTEGER,
    receipt INTEGER,
    card_no INTEGER,
    store INTEGER,
    branch INTEGER,
    match_id INTEGER,
    trans_id INTEGER)''')

NameError: name 'cur' is not defined

You can commit that table if you want. Just hit the "a" key when you're in command mode (hit escape to make _sure_ you're in command mode if you don't know the difference between command and edit modes.) Then just type in the commit command and you'll see the table. 

Now let's fill up the table with a few dozen transaction rows just to see it in action. We are going to use our file reader we used above.

In [10]:
with open(working_dir + input_file,'r') as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO transactions (datetime,register_no,emp_no,
              trans_no,upc,description,trans_type,trans_subtype,
              trans_status,department,quantity,Scale,cost,unitPrice,
              total,regPrice,altPrice,tax,taxexempt,foodstamp,wicable,
              discount,memDiscount,discountable,discounttype,voided,percentDiscount,
              ItemQtty,volDiscType,volume,
              VolSpecial,mixMatch,matched,memType,staff,numflag,itemstatus,tenderstatus,
              charflag,varflag,batchHeaderID,local,organic,display,receipt,
              card_no,store,branch,match_id,trans_id)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?)''', line)

        
        if idx > 100 :
            break

db.commit()

Now go check out the database and you'll see our table with the transactions in there. But you'll also see the headers. We want to skip that row and we can use the `next` function to do it.

In [13]:
with open(working_dir + input_file,'r',encoding="Latin-1") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO transactions (datetime,register_no,emp_no,
              trans_no,upc,description,trans_type,trans_subtype,
              trans_status,department,quantity,Scale,cost,unitPrice,
              total,regPrice,altPrice,tax,taxexempt,foodstamp,wicable,
              discount,memDiscount,discountable,discounttype,voided,percentDiscount,
              ItemQtty,volDiscType,volume,
              VolSpecial,mixMatch,matched,memType,staff,numflag,itemstatus,tenderstatus,
              charflag,varflag,batchHeaderID,local,organic,display,receipt,
              card_no,store,branch,match_id,trans_id)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?)''', line)

        
        if idx > 100 :
            break

db.commit()

Play around running those cells and the create-table cell until you get a clean one hundred rows. Now let's write a query and see what we get back.

In [ ]:
result = cur.execute('''SELECT card_no,
                               date(datetime) as date,
                               sum(total) AS spend
                        FROM transactions
                        WHERE trans_type = "I"
                        GROUP BY card_no, date
                        ''')
print(list(result)) # Note that we need to call list to *force* Python to evaluate the iterable here and print it.

Okay, that was fun. Now let's run it for real, filling up the table with our entire file. We'll start from scratch here so we get a clean table.

In [ ]:
cur.execute('''DROP TABLE IF EXISTS transactions''')
cur.execute('''CREATE TABLE transactions (
    datetime TIMESTAMP,
    register_no INTEGER,
    emp_no INTEGER,
    trans_no INTEGER,
    upc INTEGER,
    description TEXT,
    trans_type TEXT,
    trans_subtype TEXT,
    trans_status TEXT,
    department INTEGER,
    quantity REAL,
    Scale INTEGER,
    cost REAL,
    unitPrice REAL,
    total REAL,
    regPrice REAL,
    altPrice REAL,
    tax INTEGER,
    taxexempt INTEGER,
    foodstamp INTEGER,
    wicable INTEGER,
    discount REAL,
    memDiscount REAL,
    discountable INTEGER,
    discounttype INTEGER,
    voided INTEGER,
    percentDiscount REAL,
    ItemQtty REAL,
    volDiscType INTEGER,
    volume INTEGER,
    VolSpecial REAL,
    mixMatch INTEGER,
    matched INTEGER,
    memType INTEGER,
    staff INTEGER,
    numflag INTEGER,
    itemstatus INTEGER,
    tenderstatus INTEGER,
    charflag TEXT,
    varflag INTEGER,
    batchHeaderID INTEGER,
    local INTEGER,
    organic INTEGER,
    display INTEGER,
    receipt INTEGER,
    card_no INTEGER,
    store INTEGER,
    branch INTEGER,
    match_id INTEGER,
    trans_id INTEGER)''')

with open(working_dir + input_file,'r') as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO transactions (datetime,register_no,emp_no,
              trans_no,upc,description,trans_type,trans_subtype,
              trans_status,department,quantity,Scale,cost,unitPrice,
              total,regPrice,altPrice,tax,taxexempt,foodstamp,wicable,
              discount,memDiscount,discountable,discounttype,voided,percentDiscount,
              ItemQtty,volDiscType,volume,
              VolSpecial,mixMatch,matched,memType,staff,numflag,itemstatus,tenderstatus,
              charflag,varflag,batchHeaderID,local,organic,display,receipt,
              card_no,store,branch,match_id,trans_id)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?)''', line)

        #if idx > 100 :
        #    break

db.commit()

And now let's run our query again and print out the results in a prettier fashion.

In [ ]:
result = cur.execute('''SELECT card_no,
                               date(datetime) as date,
                               sum(total) AS spend
                        FROM transactions
                        WHERE trans_type = "I"
                        GROUP BY card_no, date
                        ''')

In [ ]:
for idx,row in enumerate(result) :
    print("On {0}, card_no = {1} spent {2:,.2f}.".format(row[0],row[1],row[2]))
    if idx > 20 :
        break

If you play around with this you'll notice that it keeps grabbing new rows. This is because the cursor, like the file reader, is iterating over the results set returned from the DB. 

Again, because this table is unusally complicated, I'm going to hook you up with a complicated query written by a former student, Jason Kolberg, who is great with SQL.

In [ ]:
result = cur.execute('''SELECT card_no,
                                   department,
                                   substr(date(datetime),1,4) AS year,
                                   substr(date(datetime),6,2) AS month,
                                   sum(total) AS spend,
                                   count(distinct(date(datetime) || register_no ||
                                           emp_no || trans_no)) as Transactions,
                                   sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                                   FROM transactions
                                   WHERE department != 0 and
                                        department != 15 and
                                        trans_status != 'M' and
                                        trans_status != 'C' and
                                        trans_status != 'J' and
                                       (trans_status = '' or 
                                        trans_status = ' ' or 
                                        trans_status = 'V' or 
                                        trans_status = 'R')
                          GROUP BY card_no, department, year, month''')

In [ ]:
for idx,row in enumerate(result) :
    print(row)
    if idx > 10 :
        break

Play around with the query and the results and see if you can figure out what is happening with this query. Now let's close this DB. It's a good idea to save it, because it gives you a place to test queries.

In [ ]:
db.close()

## Building a DB in Memory
This section sounds really complicated, but it's actually not too bad. You may find as you're working with the Wedge data that you need to run lots of queries against different transaction files. It's a hassle to keep putting these on a DB that exists on disc just to overwrite it on the next iteration. The following cell does everything we just did, but builds the DB in memory.

In [ ]:
#%%timeit # uncomment to test the execution time of the cell
db = sqlite3.connect(':memory:') # this is the only change!
cur = db.cursor()
    
cur.execute('''DROP TABLE IF EXISTS transactions''')
cur.execute('''CREATE TABLE transactions (
    datetime TIMESTAMP,
    register_no INTEGER,
    emp_no INTEGER,
    trans_no INTEGER,
    upc INTEGER,
    description TEXT,
    trans_type TEXT,
    trans_subtype TEXT,
    trans_status TEXT,
    department INTEGER,
    quantity REAL,
    Scale INTEGER,
    cost REAL,
    unitPrice REAL,
    total REAL,
    regPrice REAL,
    altPrice REAL,
    tax INTEGER,
    taxexempt INTEGER,
    foodstamp INTEGER,
    wicable INTEGER,
    discount REAL,
    memDiscount REAL,
    discountable INTEGER,
    discounttype INTEGER,
    voided INTEGER,
    percentDiscount REAL,
    ItemQtty REAL,
    volDiscType INTEGER,
    volume INTEGER,
    VolSpecial REAL,
    mixMatch INTEGER,
    matched INTEGER,
    memType INTEGER,
    staff INTEGER,
    numflag INTEGER,
    itemstatus INTEGER,
    tenderstatus INTEGER,
    charflag TEXT,
    varflag INTEGER,
    batchHeaderID INTEGER,
    local INTEGER,
    organic INTEGER,
    display INTEGER,
    receipt INTEGER,
    card_no INTEGER,
    store INTEGER,
    branch INTEGER,
    match_id INTEGER,
    trans_id INTEGER)''')

with open(working_dir + input_file,'r') as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO transactions (datetime,register_no,emp_no,
              trans_no,upc,description,trans_type,trans_subtype,
              trans_status,department,quantity,Scale,cost,unitPrice,
              total,regPrice,altPrice,tax,taxexempt,foodstamp,wicable,
              discount,memDiscount,discountable,discounttype,voided,percentDiscount,
              ItemQtty,volDiscType,volume,
              VolSpecial,mixMatch,matched,memType,staff,numflag,itemstatus,tenderstatus,
              charflag,varflag,batchHeaderID,local,organic,display,receipt,
              card_no,store,branch,match_id,trans_id)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?)''', line)
    
db.commit() # Best practice, but not required here

result = cur.execute('''SELECT card_no,
                                   department,
                                   substr(date(datetime),1,4) AS year,
                                   substr(date(datetime),6,2) AS month,
                                   sum(total) AS spend,
                                   count(distinct(date(datetime) || register_no ||
                                           emp_no || trans_no)) as Transactions,
                                   sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                                   FROM transactions
                                   WHERE department != 0 and
                                        department != 15 and
                                        trans_status != 'M' and
                                        trans_status != 'C' and
                                        trans_status != 'J' and
                                       (trans_status = '' or 
                                        trans_status = ' ' or 
                                        trans_status = 'V' or 
                                        trans_status = 'R')
                          GROUP BY card_no, department, year, month''')

for idx,row in enumerate(result) :
    print(row)
    if idx > 10 :
        break
        
db.close()

It may be hard to tell, but this runs about 33% faster. Plus, no annoying files to clean up. 